In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyp
import seaborn as sns
%matplotlib inline

C:\Users\IMOE001\AppData\Local\Temp\ipykernel_13324\163558748.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#phishing_url_df = pd.read_csv("../Input/data_phishing_37175.csv")
col_names = ["index", "url"]
#phishing_url_df.columns = col_names

#phishing_url_df = list(phishing_url_df["url"])
#first_50_phishing_url_df = phishing_url_df[:] 
#print(first_50_phishing_url_df)

legitimate_url_df = pd.read_csv("../Input/data_legitimate_36400.csv")
legitimate_url_df.columns = col_names
legitimate_urls = list(legitimate_url_df["url"])


In [4]:
import domain_parser

#parsed_urls = domain_parser.parse(first_50_phishing_url_df, "phishing")
#print(parsed_urls)

parsed_urls = domain_parser.parse(legitimate_urls, "legitimate")

In [5]:
from dataPreProcessing import filter_brands
from dataPreProcessing import filter_keywords
from dataPreProcessing import subtract_lists
from dataPreProcessing import filter_randoms
from dataPreProcessing import filter_composes
from dataPreProcessing import brand_names
from dataPreProcessing import keywords
from dataPreProcessing import filter_maliciouses

for parsed_url in parsed_urls:
    parsed_url["words_raw_for_filters"] = list(parsed_url["words_raw"]) # remove from all dicts at the end
    
    # brands
    parsed_url["brand_words"] = filter_brands(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["brand_words"])

    # keywords
    parsed_url["keyword_words"] = filter_keywords(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["keyword_words"])

    # randoms
    parsed_url["random_words"] = filter_randoms(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["random_words"])

    # filter word less then 7 letters
    parsed_url["word_list"] = []
    for word in list(parsed_url["words_raw_for_filters"]):
        if len(word) <= 7:
            parsed_url["word_list"].append(word)
            parsed_url["words_raw_for_filters"].remove(word)
    
    # try to decompose the rest
    parsed_url["word_composed"] = []
    for word, composed in filter_composes(parsed_url["words_raw_for_filters"]):
        for internal_word in composed:
            if word in brand_names():
                parsed_url["brand_words"].append(internal_word)
            
            elif word in keywords():
                parsed_url["keyword_words"].append(internal_word)
            
            else:
                parsed_url["word_list"].append(internal_word)
        
        parsed_url["word_composed"].append(word)
        parsed_url["words_raw_for_filters"].remove(word)

    
    # the rest is non brands, non keywords, non randoms, with len more then 7 chars
    # and no composed.
    for word in list(parsed_url["words_raw_for_filters"]):
        parsed_url["word_list"].append(word)
        parsed_url["words_raw_for_filters"].remove(word)
    

    # check the rest for maliciousness
    parsed_url["word_malicious"] = [mal_word for (mal_word, _, _) in filter_maliciouses(parsed_url["word_list"])]
    parsed_url["word_list"] = subtract_lists(parsed_url["word_list"], list(set(parsed_url["word_malicious"])))


    #now can remove the helper list 'words_raw_for_filters'
    parsed_url.pop('words_raw_for_filters')



#print(parsed_urls)

In [6]:
print(len(parsed_urls))

36400


In [7]:
import json

#with open("../Input/preprocess_phishing_37175.json", "w") as f:
#    json.dump(parsed_urls, f)

with open("../Input/preprocess_legitimate_36400.json", "w") as f:
    json.dump(parsed_urls, f)


In [10]:
with open("../Input/preprocess_legitimate_36400.json", "r") as f:
    l = json.loads(f.read())
print(len(l))

with open("../Input/preprocess_phishing_37175.json", "r") as f:
    l2 = json.loads(f.read())
print(len(l2))



36400
37175
